# setting up cities MySQL database(s) on AWS
Goal is: setting up one or more databases with cities data. The cities data we want are:
- name
- country
- country code
- population
- year of population measurement
- latitude
- longitude

The cities database(s) will be static databases as their information won't change on a daily basis. Any change to cities in the databases can/ will be done manually.

## web-scrapping
In this notebook we'll be web-scrapping cities information from Wikipedia pages although web-scrapping might not be reliable (or desirable). We use web-scrapping here as part of learning how to do this and in another notebook we'll be using API calls to gather our data. 

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
from src.wikipedia.get_city_info import get_city_info
from src.service.mysql_db import MySQL

In [4]:
# top 5 largest cities in Germany
city_names = ['Berlin']

In [5]:
# get cities information
cities_info_df = pd.DataFrame([get_city_info(city_name) for city_name in tqdm(city_names)])
cities_info_df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

,city_name,country,country_code,population,measurement_year,latitude,longitude
0,Berlin,Germany,DE,3850809,2021,52.52,13.405


In [6]:
# initialize MySQL class
con = MySQL()

In [7]:
# write to cities table
cities_info_df[['city_name', 'country', 'country_code']].to_sql('cities', if_exists='append', con=con.con(), index=False)

1

In [8]:
# get cities df for unique city id
cities_df = pd.read_sql_table('cities', con=con.con())
cities_df.head()

,city_id,city_name,country,country_code
0,1,Berlin,Germany,DE


In [9]:
# add assigned unique id's to cities_info_df
cities_info_df = cities_info_df.merge(cities_df[['city_id', 'city_name']], how='left')
cities_info_df.head()

,city_name,country,country_code,population,measurement_year,latitude,longitude,city_id
0,Berlin,Germany,DE,3850809,2021,52.52,13.405,1


In [10]:
# create populations df with city_id from SQL server
cities_info_df[['city_id', 'population', 'measurement_year']].to_sql('populations', if_exists='append', con=con.con(), index=False)

1

In [11]:
# create populations df with city_id from SQL server
cities_info_df[['city_id', 'latitude', 'longitude']].to_sql('cities_location', if_exists='append', con=con.con(), index=False)

1